In [1]:
import pickle
a = open("Articles.pickle",'rb')
uncleanedarticles = pickle.load(a)
articles=[]
text=""
title=""
titles=[]
sources=[]
for i in uncleanedarticles:
    source,url,title,text = i[0],i[1],i[2],i[3]
    sources.append(source)
    text = text.replace('\n\n','\n')
    if text =="":
        continue
#     article = i.split('\n\n\n\n\n')
#     text = article[1].strip('\n').strip(' ')
    if text.count('\n') <10:
        continue
    if text.count('https://') >10:
        continue   
#     title = article[0].strip('\n').strip(' ')
    title = title.strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([source,url,title,text])
a.close()
# print(articles[0])

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}

articleslen=[]
for i in articles:
    title,text = i[2],i[3]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [3]:
# print(len(wordRepArticles))
# print(wordRepArticles[0])
print(len(articles))
print(len(uncleanedarticles))

2352
6198


In [4]:
articles[:3]

[['abcnews',
  'https://abcnews.go.com/US/reads-monday/story?id=59712958',
  'Must reads before Monday',
  'One parent\'s decision to shame his daughter on social media drew mixed responses, Florida is struggling with major toxic algae blooms, the only operating spaceport in the world is crumbling, and eight children affected by Trump\'s "zero-tolerance" border policy are still waiting to be reunited with their families.\nDon’t miss out on some of ABC News\' best reporting from this week.\n\'What are you trying to prove?\': Parenting experts weigh in on dad\'s bullying punishment\nSTOCK/Getty Images\nA viral video posted by a father that forced his daughter to walk 5 miles to school in near-freezing temperatures has prompted questions about the use of social media as a way to humiliate and discipline children. Some experts say that shame is not an "effective way to change behavior."\n\'When is this going to stop?\': Florida activist highlights devastating impact of algae blooms on wild

In [5]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(n):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,n):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [6]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][2])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        if i<50:
            print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][2])

Main title: Must reads before Monday
0/3: Father denied visa for slain daughter's funeral as suspect charged with rape, murder
1/3: Man accused of killing 13-year-old Hania Aguilar could face death penalty if convicted
2/3: FBI: Man arrested in death of North Carolina teenager
Main title: Worcester firefighter dies while battling huge blaze, the latest in a string of firefighter deaths in city
0/3: Chief identifies firefighter who died after becoming trapped in five-alarm Worcester fire
1/3: 5 children under age of 10 die in Ohio house fire
2/3: California marks 1 month since deadly wildfires ripped through state
Main title: Funeral for 'Baby Jane Doe' found near California highway
0/3: Parents charged in 10-month-old daughter's overdose death
1/3: Texas Daycare Worker Accused of Breaking Baby’s Bones
2/3: Boyfriend allegedly kills girlfriend and her 17-year-old daughter: Sheriff
Main title: Amazon touted as big win for NY, but math is more complex
0/3: CVS offers 'guaranteed net cost'

Main title: 16 movies, TV series and more premiering Dec. 9 to 15
Main title: 2019 Grammy nominees: Kendrick Lamar, Drake, Lady Gaga, Cardi B, Taylor Swift
Main title: H.E.R., Ella Mai, Mendes and more react to the Grammy noms
Main title: 25 Days of Cookies: Jamie Oliver's 'Billionaire Shortbread' cookie recipe
Main title: 25 Days of Cookies: Reese Witherspoon's 'Cowboy Cookies' recipe
Main title: 25 Days of Cookies: Cookie Monster's Me Double Chocolate cookie recipe
Main title: How to find the perfect dress for any and every type of wedding: From a destination to a City Hall ceremony
Main title: How the new tax code actually helps your side hustle!
Main title: 'Other,' and the power of 'The Good Doctor'
Main title: Blind USC senior walks the football field for the last time as a Trojan
Main title: New parents call out Amazon for slipping sponsored products onto baby registries without them knowing
Main title: NICU nurses donate $10K Mega Millions winnings to colleagues in need
Main ti

Main title: Sandy Hook school shooter had 'scorn for humanity,' according to newly released documents
Main title: Anti-Semitic pamphlets found in Pittsburgh synagogue shooting neighborhood
Main title: Thomas Markle shares emotional notes from estranged daughter Duchess Meghan
Main title: James Comey: Russia investigation started with four Americans with ties to Trump, Russia
Main title: Who is Isha Ambani, and why are Beyoncé and Hillary Clinton in India for her wedding?
Main title: 25 awesome gifts men actually want this year
Main title: Retirement planning: Improving 401(k) plans for workers who invest
Main title: Social Security notices showing cost-of-living increases available online for retirees
Main title: Holiday gifts 2018: Best investing books for presents
Main title: Here's what happens to unsold Christmas trees
Main title: Dolphins stun Patriots with Kenyan Drake's last-second touchdown after two laterals
Main title: Admiral Schofield's huge game rallies No. 7 Tennessee to 

Main title: Florida's 2020 election: Deflated Democrats have hope. So does the GOP
Main title: Orange is the new blue: California Democrats sweep 7 House seats in former GOP stronghold
Main title: Florida recount: Sen. Bill Nelson concedes following manual recount; Rick Scott heads to Senate
Main title: Florida recount: Things smoother than in 2000, but fixes needed; Gillum ends campaign
Main title: 'This was not just about an election cycle': Andrew Gillum concedes in Florida governor's race
Main title: Florida recount: Here is why Sen. Bill Nelson's chances of winning reelection are slim
Main title: Ethics panel rebukes prominent Republican lawmaker over response to aide's sexual harassment
Main title: 'Awakening': Abigail Spanberger's House win illustrates the shift of US politics in 'year of the badass woman'
Main title: Katie Porter claims Democrats' latest U.S. House victory in California
Main title: Alexandria Ocasio-Cortez's first days in Congress: Treated as intern, clothing c

Main title: Prosecutors: Chicago cops betrayed oath in covering up the shooting death of Laquan McDonald
Main title: Don't let Mitch McConnell foil this smart, bipartisan prison reform
Main title: Police: Emantic Bradford's gun 'heightened the sense of threat' at Ala. mall before he was shot
Main title: Shutting former criminals out of opportunity is economically foolish
Main title: Criminal justice reform: We can improve expensive, ineffective system by lowering recidivism
Main title: Chicago cops accused of covering up Laquan McDonald shooting to go to trial
Main title: Florida felons' restored voting rights will have national, personal impacts
Main title: Jemel Roberson: Family demands police release name of cop who shot black security guard
Main title: Florida finally got it right on voting for ex-felons. It's not just about rights, it's about public health
Main title: Lame-duck Congress' rush for criminal justice reform plan will hurt, not help: Tom Cotton
Main title: Midterms sho

...

Main title: Japan's most beautiful ryokans encourage you to do nothing
Main title: AB Concept of Hong Kong delivers stunning hotel interiors
Main title: The world's sexiest bedrooms, according to Mr & Mrs Smith
Main title: Grace Millane: Man appears in court charged with backpacker's murder
Main title: Brexit: Critics urge renegotiation as ministers say vote 'going ahead'
Main title: Yellow vest protests 'economic catastrophe' for France
Main title: Charlie Rowley: Novichok victim 'terrified' about health
Main title: I'm a Celebrity: Harry Redknapp wins ITV series
Main title: NHS told to ditch 'absurd' fax machines
Main title: O2 'to seek millions' in damages over data outage
Main title: Khashoggi murder: Saudis refuse Turkey extradition request
Main title: Raheem Sterling: Manchester City forward says newspapers 'fuel racism' in football
Main title: Copa Libertadores final: River Plate beat Boca Juniors 3-1 in extra time (agg 5-3)
Main title: Doctor Who: Jodie Whittaker won't return f

Main title: Krishnamoorthi: Comey 'Came Across as Consistent, Compelling and Careful'
Main title: Adam Schiff: 'Very Real Prospect' Donald Trump Faces 'Jailtime'
Main title: Nick Cannon calls out Sarah Silverman, Chelsea Handler, Amy Schumer for old homophobic tweets
Main title: 'Game of Thrones' The Mountain star Hafthor Bjornsson talks finale, spinoffs and being World's Strongest Man
Main title: 'He Thinks They're Dumb': Hegseth Blasts Chuck Todd for Claiming Trump Supporters 'Want to Be Gullible'
Main title: Florida man jailed for using secretly-recorded sex videos for porn website
Main title: Want to help a veteran suffering from PTSD? Here’s how
Main title: Here's how to overcome holiday anxiety and stress
Main title: France protests and the 'yellow vests’ deep anger reveal the hypocrisy at the heart of the green agenda
Main title: Former Homeland Security Secretary Tom Ridge: Trump was right to withdraw from Iran deal
Main title: Why are so many people determined to suck all the 

Main title: In the era of endless robocalls, why telemarketers persist
Main title: France's finance minister warns of catastrophic effects of 'yellow jacket' protests on the economy
Main title: How money stokes divide of historic black community in Virginia pipeline battle
Main title: 'Kids are falling off': Why fewer children have health insurance now
Main title: Michael Cohen filings renew discussion of Trump's impeachment
Main title: Comey says he launched probe into Clinton email leaks before he was fired
Main title: Sen. Angus King: Not enough evidence against Trump yet for impeachment
Main title: Trump says John Kelly to leave chief of staff role by year end
Main title: On film and TV sets, intimacy coordinators focus on safety for actors in sex scenes
Main title: Even without proof, CBD is finding a niche as a cure-all
Main title: 'Tariff man' Trump has shifted Republicans toward a pro-trade stance
Main title: China summons U.S. ambassador, warns Canada of 'grave consequences' i

Main title: Trump addresses U.N., brags about his accomplishments, draws derisive laughter
Main title: Iran terrorist attack shows regime opponents see 'unique opportunity,' experts say
Main title: Teen survives 49 days adrift in Pacific, is rescued off Guam
Main title: Some young Catholics shun modern life, embrace religion in U.K.
Main title: China warns U.S. to withdraw sanctions or 'bear the consequences'
Main title: Turkey's Erdogan to build more prisons as post-coup purge persists
Main title: As Israel moves right, young American Jews target Birthright tours in protest
Main title: Ireland takes aim at prenatal alcohol exposure
Main title: Bees are dying at an alarming rate. Amsterdam may have the answer.
Main title: China eases economic pressure on North Korea, undercutting the Trump admin
Main title: How Europe plans to skirt Trump's sanctions and keep doing business with Iran
Main title: Despite Trump's promise to protect them, Christian refugees struggle to enter U.S.
Main tit

Main title: This Robotruck’s Cameras See Twice as Far as Any Lidar
Main title: An Energy Evolution: From Delicious to Dirty to Almost Free
Main title: Trump’s Limits on Legal Immigration Could Hit Businesses Hard
Main title: How the CIA Used a Fake Sci-Fi Flick to Rescue Americans From Tehran
Main title: It May Be the Right Time to Change Your Trading Strategy
Main title: Analysts Downplay Qualcomm's Short-Term Legal Victory Over Apple
Main title: Apple Shareholders Have Larger Legal Issues Than Qualcomm to Consider
Main title: Kass: Apple's Share Price May Still Be Vulnerable
Main title: Legal Quarrel With Qualcomm Adds to Apple Shareholder Anxiety Over China
Main title: Apple Stock's Upside Potential Tamped Down by Tightening Trade Tensions
Main title: Costco Is Set to Make a Play for the Upside, Stay Long and Buy Strength
Main title: Stocks End Day in the Green as Dow, S&P Overcome Slow Starts
Main title: A Major Crack Just Opened in the S&P 500
Main title: Apple and Tesla: Jim Cram

In [7]:
print(avgArticleLen)

972.2436224489796


In [8]:
sentmatch = []
for i in range(n):
    mainarticle = wordRepArticles[i]
    rep=[]
    for j in range(matches):
        comparearticle = wordRepArticles[topmatches[i][j][1]]
        bestmatchSentence =[]
        for k in range(len(mainarticle[2])):
            val = -1
            msentence = mainarticle[2][k][1]
            mainlength = len(mainarticle[2][k][0].split(' '))
            for l in range(len(comparearticle[2])):
                csentence = comparearticle[2][l][1]
                comparelength = len(comparearticle[2][l][0].split(' '))
                commonwords = list(set(msentence.keys()) & set(csentence.keys())) 
                value = 0
                for word in commonwords:
                    mainvalue = (1+math.log(msentence[word]))*1.0/(math.log(1+mainlength/avgArticleLen))
                    compvalue = (1+math.log(csentence[word]))*1.0/(math.log(1+comparelength/avgArticleLen))
                    value += mainvalue*compvalue* math.log(1+n/idf[word])
                if value>val:
                    val = value
                    ind = l
            bestmatchSentence.append([ind,val,comparearticle[2][ind][0]])
        rep.append(bestmatchSentence)
    sentmatch.append(rep)

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 

sources = list(set(sources))

reliability = [9.9 for i in range(len(sources))]
articleaggreeement=[]
for i in range(n):
    articleval =0
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        val=0
        comparticle=wordRepArticles[topmatches[i][j][1]]
        for k in range(len(mainarticle[2])):
            sent1 = analyzer.polarity_scores(mainarticle[2][k][0])['compound']
            sent2 = analyzer.polarity_scores(sentmatch[i][j][k][2])['compound']
            if (sent1 >0.05 and sent2 >0.05) or (sent1 <-0.05 and sent2 <-0.05) or (sent1 <0.05 and sent2 <0.05 and sent1 >-0.05 and sent2 >-0.05):
                val +=1
#             elif 
#             val +=sent1*sent2
        val /= len(mainarticle[2])
        topmatches[i][j][0] = val*100
        articleval +=val*reliability[sources.index(articles[topmatches[i][j][1]][0])]
    articleaggreeement.append(articleval/matches)
    articles[i].append(articleval/matches)

for i in range(n):
    articles[i][0] = sources.index(articles[i][0])
    articles[i].append(i)

In [10]:
print(articles[:2])
print(topmatches[:2])

[[14, 'https://abcnews.go.com/US/reads-monday/story?id=59712958', 'Must reads before Monday', 'One parent\'s decision to shame his daughter on social media drew mixed responses, Florida is struggling with major toxic algae blooms, the only operating spaceport in the world is crumbling, and eight children affected by Trump\'s "zero-tolerance" border policy are still waiting to be reunited with their families.\nDon’t miss out on some of ABC News\' best reporting from this week.\n\'What are you trying to prove?\': Parenting experts weigh in on dad\'s bullying punishment\nSTOCK/Getty Images\nA viral video posted by a father that forced his daughter to walk 5 miles to school in near-freezing temperatures has prompted questions about the use of social media as a way to humiliate and discipline children. Some experts say that shame is not an "effective way to change behavior."\n\'When is this going to stop?\': Florida activist highlights devastating impact of algae blooms on wildlife\nJoe Rae

In [11]:
sourcerelia=[]
for i in range(len(sources)):
    sourcerelia.append([i,sources[i],reliability[i]])
fileObject = open('./mysite/sourcereliability.pickle','wb')
pickle.dump(sourcerelia,fileObject)
fileObject.close()
print(sourcerelia[0])

[0, 'washingtonpost', 9.9]


In [12]:
articlesimilarity=[]
for i in range(n):
    for j in range(len(topmatches[i])):
#         articlesimilarity.append([articles[i][2],articles[topmatches[i][j][1]][2]])
        articlesimilarity.append([i,topmatches[i][j][1],topmatches[i][j][0]])
fileObject = open('./mysite/articlessimilarity.pickle','wb')
pickle.dump(articlesimilarity,fileObject)
fileObject.close()

In [13]:
print(articlesimilarity[0])

[0, 400, 64.28571428571429]


In [14]:
fileObject = open('./mysite/cleanedarticles.pickle','wb')
pickle.dump(articles,fileObject)
fileObject.close()

In [ ]:
# max([i[0] for i in articles])

In [ ]:
topmatches[0]